In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import time

/Users/mustafamuratarat/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_train=pd.read_csv('/Users/mustafamuratarat/Desktop/train/X_train.txt', header=None).as_matrix()
y_train = pd.read_csv('/Users/mustafamuratarat/Desktop/train/y_train.txt', header=None).as_matrix().ravel()
X_test = pd.read_csv('/Users/mustafamuratarat/Desktop/test/X_test.txt', header=None).as_matrix()
y_test = pd.read_csv('/Users/mustafamuratarat/Desktop/test/y_test.txt', header=None).as_matrix().ravel()

In [3]:
def convertDummy(y, labelNum):
    labelNum = tf.constant(labelNum)
    dummy = tf.one_hot(y, labelNum, axis=1)
    sess = tf.Session()
    dummy = sess.run(dummy)
    sess.close()
    return dummy

In [4]:
def length(sequence):
  used = tf.sign(tf.reduce_max(tf.abs(sequence), 2))
  length = tf.reduce_sum(used, 1)
  length = tf.cast(length, tf.int32)
  sess = tf.Session()
  length = sess.run(length)
  sess.close()
  return length

In [5]:
max_timestep = 60
num_classes = 6
split_size = max_timestep

In [6]:
 def split_time(m, split_size):
        r = m.shape[0]
        extend_row_size = np.math.ceil(r / split_size) * split_size - r
        m_p = np.expand_dims(np.pad(m, [(0, extend_row_size), (0, 0)], mode='constant'), axis=0)
        result = m_p.reshape((np.math.ceil(r / split_size), split_size, m.shape[1]))
        return result

In [7]:
Xtrain = split_time(X_train, split_size)
ytrain = split_time(convertDummy(y_train-1, num_classes), split_size)
Xtest = split_time(X_test, split_size)
ytest = split_time(convertDummy(y_test-1, num_classes), split_size)

In [8]:
print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

(123, 60, 561)
(123, 60, 6)
(50, 60, 561)
(50, 60, 6)


In [19]:
length(Xtrain) #the last instance has only 32 time steps. We left-pad them with zeroes.

array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60,
       60, 60, 60, 32], dtype=int32)

In [9]:
seq_length_batch = length(Xtrain)

n_neurons = 128
num_inputs = 561
num_steps=60
X = tf.placeholder(tf.float32, [None, num_steps, num_inputs])
seq_length = tf.placeholder(tf.int32, [None])

initializer = tf.random_uniform_initializer(-1, 1)
cell = tf.contrib.rnn.LSTMCell(num_units=n_neurons, state_is_tuple=True, forget_bias=1.0, initializer=initializer)
outputs, states= tf.nn.dynamic_rnn(cell, X, sequence_length=seq_length, dtype=tf.float32)


init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X: Xtrain, seq_length:seq_length_batch})

In [10]:
print(outputs_val.shape)

(123, 60, 128)


In [11]:
#state: if state_is_tuple is False, this must be a state Tensor, 2-D, batch x state_size. 
#If state_is_tuple is True, this must be a tuple of state Tensors, both 2-D, with column sizes c_state and m_state.
#c_state and m_state are "Memory State" and "Cell State". (m_state has the same meaning as h_state)

#Those tensors represent combined internal state of the cell, and should be passed together. 
#Old way to do it was to simply concatenate them, and new way is to use tuples.

#c = (sigmoid(f + self._forget_bias) * c_prev + sigmoid(i) * self._activation(j))

#and

#m = sigmoid(o) * self._activation(c)

#and 

#new_state = (LSTMStateTuple(c, m) 

#If you compare the code to compute c and m with the LSTM equations, you can see it corresponds to the cell state 
#(typically denoted with c) and hidden state (typically denoted with h), respectively.

#new_state = (LSTMStateTuple(c, m) indicates that the first element of the returned state tuple is c 
#(cell state a.k.a. c_state), and the second element of the returned state tuple is m (hidden state a.k.a. m_state).
type(states_val)

print(states_val[0]) #new cell state
print(states_val[1]) #new hidden state

#Moreover the new hidden state tuple contains the final state of each cell (look below)

[[-2.2261581e+01 -5.9861191e+01  6.1622041e-01 ... -5.0424500e+01
  -5.2498875e+01  1.8757626e-05]
 [-6.2237778e+00 -5.8858715e+01  3.1848245e+00 ... -1.5879152e+01
  -8.4553318e+00  2.1313832e-03]
 [-2.5817682e+01 -5.8589535e+01  4.0715318e+00 ... -6.5649724e+00
  -3.2205780e+01 -8.3018822e-06]
 ...
 [-1.0220116e+00 -5.8531055e+01  1.6158296e-03 ... -2.3929796e+01
  -4.7286873e+01  1.6072131e-04]
 [-4.1211290e+00 -5.9858986e+01  8.3531780e+00 ... -5.1728897e+00
  -1.1686237e+00  2.3650814e-06]
 [-2.0673704e+01 -3.1958836e+01  1.1888981e+01 ... -2.1217928e+00
  -5.2904539e+00  8.0984551e-03]]
[[-1.0000000e+00 -9.9993694e-01  1.9291642e-06 ... -7.7934231e-04
  -9.9980611e-01  1.8592676e-05]
 [-9.9998212e-01 -9.9244648e-01  3.8856086e-01 ... -1.4057340e-01
  -9.7687006e-01  2.0994036e-03]
 [-1.0000000e+00 -2.6862469e-01  1.8426839e-03 ... -9.2686445e-04
  -9.9994504e-01 -8.3007662e-06]
 ...
 [-7.7068466e-01 -9.9701047e-01  4.4745425e-09 ... -4.4300382e-06
  -9.9999022e-01  1.3935380e-04]

In [12]:
print(outputs_val[122][32])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [13]:
#final state for the first instance
states_val[1][0]

array([-1.0000000e+00, -9.9993694e-01,  1.9291642e-06,  7.9703939e-05,
        1.7265876e-01,  9.1469674e-06, -4.9654335e-02, -7.6785994e-01,
        9.9589288e-01,  7.5930792e-01,  1.0776148e-12, -6.7408052e-03,
       -1.3225140e-04, -2.0024334e-03, -1.8743804e-03,  7.2587636e-07,
       -3.1778383e-01,  8.6504014e-17, -1.5321133e-06, -6.3138187e-01,
       -1.0000000e+00, -3.4901130e-04,  2.9981215e-04, -8.6333588e-05,
        2.5757418e-08, -1.5266814e-04, -2.4254780e-09, -9.9911296e-01,
        9.9724603e-01, -9.9831003e-01,  6.1948068e-04, -2.7498906e-18,
        7.6127100e-01, -1.3139486e-06, -4.4278200e-05,  3.2466814e-02,
       -7.6171380e-01,  2.3750241e-07, -8.2052160e-05, -7.6219475e-01,
       -3.3910876e-09, -9.8948812e-01,  3.2182992e-09, -2.3111209e-12,
       -9.7669872e-06,  7.1865087e-04, -6.0348025e-06,  9.9574941e-01,
        1.0476566e-01, -2.7343239e-10, -9.9996245e-01,  1.1412669e-09,
       -9.9997950e-01, -9.5248595e-07,  1.6790977e-02,  9.3638444e-01,
      

In [14]:
#output of the final time steps.THEY ARE THE SAME!
outputs_val[0][59]

array([-1.0000000e+00, -9.9993694e-01,  1.9291642e-06,  7.9703939e-05,
        1.7265876e-01,  9.1469674e-06, -4.9654335e-02, -7.6785994e-01,
        9.9589288e-01,  7.5930792e-01,  1.0776148e-12, -6.7408052e-03,
       -1.3225140e-04, -2.0024334e-03, -1.8743804e-03,  7.2587636e-07,
       -3.1778383e-01,  8.6504014e-17, -1.5321133e-06, -6.3138187e-01,
       -1.0000000e+00, -3.4901130e-04,  2.9981215e-04, -8.6333588e-05,
        2.5757418e-08, -1.5266814e-04, -2.4254780e-09, -9.9911296e-01,
        9.9724603e-01, -9.9831003e-01,  6.1948068e-04, -2.7498906e-18,
        7.6127100e-01, -1.3139486e-06, -4.4278200e-05,  3.2466814e-02,
       -7.6171380e-01,  2.3750241e-07, -8.2052160e-05, -7.6219475e-01,
       -3.3910876e-09, -9.8948812e-01,  3.2182992e-09, -2.3111209e-12,
       -9.7669872e-06,  7.1865087e-04, -6.0348025e-06,  9.9574941e-01,
        1.0476566e-01, -2.7343239e-10, -9.9996245e-01,  1.1412669e-09,
       -9.9997950e-01, -9.5248595e-07,  1.6790977e-02,  9.3638444e-01,
      

In [23]:
outputs_val[122][31]

array([-9.99996066e-01, -9.99121726e-01,  9.47958946e-01,  6.64272811e-04,
        9.97423351e-01,  1.51272346e-08, -3.43467121e-08, -8.56500342e-02,
        9.98469889e-01,  6.53836012e-01, -2.21689441e-03, -5.03772462e-04,
       -9.80075262e-03,  2.64362954e-02,  3.90944630e-02,  1.59144108e-07,
       -9.94364977e-01, -1.94143546e-09, -2.09245179e-03, -4.87343609e-01,
       -6.23615444e-01, -3.61945093e-01,  3.11713084e-05, -7.02876225e-02,
       -3.29719740e-03, -1.20037105e-02, -1.86250535e-08, -3.93456531e-07,
        9.99754131e-01, -8.94902408e-01, -2.86640761e-06,  4.72174091e-18,
        7.59040833e-01, -1.65996581e-01, -4.25734584e-07,  2.33514484e-07,
       -4.16531488e-02,  1.05623947e-03, -1.98415667e-01, -7.69106686e-01,
       -4.61503892e-04, -7.81199574e-01,  3.61764418e-09,  4.20641445e-04,
       -3.33937351e-04, -7.57226317e-08, -1.34396472e-03,  1.00000000e+00,
        9.99970675e-01,  6.21273011e-06, -2.41834685e-01,  2.01900264e-13,
       -7.44437397e-01, -

In [26]:
states_val[1][122]

array([-9.99996066e-01, -9.99121726e-01,  9.47958946e-01,  6.64272811e-04,
        9.97423351e-01,  1.51272346e-08, -3.43467121e-08, -8.56500342e-02,
        9.98469889e-01,  6.53836012e-01, -2.21689441e-03, -5.03772462e-04,
       -9.80075262e-03,  2.64362954e-02,  3.90944630e-02,  1.59144108e-07,
       -9.94364977e-01, -1.94143546e-09, -2.09245179e-03, -4.87343609e-01,
       -6.23615444e-01, -3.61945093e-01,  3.11713084e-05, -7.02876225e-02,
       -3.29719740e-03, -1.20037105e-02, -1.86250535e-08, -3.93456531e-07,
        9.99754131e-01, -8.94902408e-01, -2.86640761e-06,  4.72174091e-18,
        7.59040833e-01, -1.65996581e-01, -4.25734584e-07,  2.33514484e-07,
       -4.16531488e-02,  1.05623947e-03, -1.98415667e-01, -7.69106686e-01,
       -4.61503892e-04, -7.81199574e-01,  3.61764418e-09,  4.20641445e-04,
       -3.33937351e-04, -7.57226317e-08, -1.34396472e-03,  1.00000000e+00,
        9.99970675e-01,  6.21273011e-06, -2.41834685e-01,  2.01900264e-13,
       -7.44437397e-01, -